# Notes to Improve Workflow

In [1]:
# Setup matplotlib to plot inline (within the notebook)
%matplotlib inline
# Import the pyplot module of Matplotlib as plt
import matplotlib.pyplot as plt
# Import pandas under the abbreviation 'pd'
import pandas as pd
# Import NumPy under the abbreviation 'np'
import numpy as np

#Notes for efficienccy:
#shift + tab when cursor is in arguments to bring up its documentation
#ex: pd.DataFrame(Shift+tab here)
#recall: Json == Dictionary
#to know functions of pandas:
dir(pd)

['BooleanDtype',
 'Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Flags',
 'Float32Dtype',
 'Float64Dtype',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NA',
 'NaT',
 'NamedAgg',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseDtype',
 'StringDtype',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__all__',
 '__builtins__',
 '__cached__',
 '__deprecated_num_index_names',
 '__dir__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__getattr__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_is_numpy_dev',
 '_libs',
 '_testing',
 '_typing',
 '_ve

# Reading The Data

In [2]:
# using pandas and read_csv()
acquis = pd.read_csv("datasets/startups/acquisitions.csv")
additions = pd.read_csv("datasets/startups/additions.csv")
companies = pd.read_csv("datasets/startups/companies.csv")
invests = pd.read_csv("datasets/startups/investments.csv")
rounds = pd.read_csv("datasets/startups/rounds.csv")
dfAll = [acquis, additions, companies, invests, rounds]

In [3]:
#shapes (rows x cols) of each dataset
for ele in dfAll:
    print(ele.shape)

(18969, 18)
(2213, 4)
(66368, 14)
(168647, 18)
(114949, 12)


# Samples of the datasets

In [4]:
acquis.sample(5).transpose()

,12423,18017,2343,15677,4148
company_permalink,/organization/parago-2,/organization/viracor-ibt-laboratories,/organization/blue-ribbon-soundworks,/organization/spinshark,/organization/cottman-transmission-holdings
company_name,parago,Viracor-IBT Laboratories,Blue Ribbon SoundWorks,SpinShark,Cottman Transmission Holdings
company_category_list,Advertising,Health Care|Health Care Information Technology...,NaN,Advertising,NaN
company_country_code,USA,USA,NaN,USA,NaN
company_state_code,TX,MO,NaN,IN,NaN
company_region,Dallas,MO - Other,NaN,Indianapolis,NaN
company_city,Lewisville,Lees Summit,NaN,Carmel,NaN
acquirer_permalink,/organization/blackhawk-network,/organization/eurofins,/organization/microsoft,/organization/pm-digital,/organization/american-capital-strategies
acquirer_name,BlackHawk Network,Eurofins Scientific,Microsoft,PM Digital,American Capital Strategies
acquirer_category_list,E-Commerce|Finance,Biotechnology,Software|Video Games,Advertising|Analytics|Creative|Displays|Intern...,Finance


In [5]:
additions.sample(5).transpose()

,1140,398,932,976,2211
content,Profiles: Products,IPOs,Profiles: Organizations,Profiles: People,Rounds: venture
month_str,2015-01,1987-06,2015-01,2010-01,2015-11
year_str,2015,1987,2015,2010,2015
value,610,9,8494,2165,2233


In [6]:
companies.sample(5).transpose()

,33642,62240,46921,32743,44879
permalink,/organization/lyra-health,/organization/visible-measures,/organization/rapidmind,/organization/live-calendars,/organization/precursor-energetics
name,Lyra Health,Visible Measures,RapidMind,Live Calendars,Precursor Energetics
homepage_url,http://www.lyrahealth.com,http://www.visiblemeasures.com,http://www.rapidmind.net,NaN,http://www.precursorenergetics.com
category_list,Information Technology,Advertising,Software,Consulting,Clean Technology
funding_total_usd,38100000,70800000,10866000,-,9500005
status,operating,operating,acquired,operating,operating
country_code,USA,USA,CAN,CAN,USA
state_code,CA,MA,ON,QC,CA
region,SF Bay Area,Boston,Toronto,Montreal,SF Bay Area
city,Burlingame,Boston,Waterloo,Laval,Santa Clara


In [7]:
invests.sample(5).transpose()

,28621,108180,151642,95696,22295
company_permalink,/organization/chase-pharmaceuticals,/organization/peachme,/organization/trusted,/organization/nanosolar,/organization/branding-brand
company_name,Chase Pharmaceuticals,Peach,Trusted,Nanosolar,Branding Brand
company_category_list,Biotechnology,Financial Services|Insurance|Software,Apps|Child Care,Clean Technology|Nanotechnology,Apps|E-Commerce Platforms|Mobile|Mobile Commer...
company_country_code,USA,USA,USA,USA,USA
company_state_code,DC,MD,CA,CA,PA
company_region,"Washington, D.C.",Baltimore,SF Bay Area,SF Bay Area,Pittsburgh
company_city,Washington,Baltimore,San Francisco,San Jose,Pittsburgh
investor_permalink,/organization/cipla,/organization/hipark,/organization/kpcb-edge,/organization/mohr-davidow-ventures,/organization/insight-venture-partners
investor_name,Cipla,HiPark,KPCB Edge,Mohr Davidow Ventures,Insight Venture Partners
investor_country_code,IND,NaN,USA,USA,USA


In [8]:
rounds.sample(5).transpose()

,85720,8984,13706,29347,36547
company_permalink,/organization/sand-2,/organization/authoria,/organization/bluestacks,/organization/duda-mobile,/organization/flow-state-media
company_name,Sand 9,Peopleclick Authoria,BlueStacks,Duda,Flow State Media
company_category_list,Mobile|Wireless,Software,Android|Apps|Mobile|Mobile Games,Curated Web|Internet|Mobile|Optimization,Games
company_country_code,USA,USA,USA,USA,USA
company_state_code,MA,MA,CA,CA,CA
company_region,Boston,Boston,SF Bay Area,SF Bay Area,SF Bay Area
company_city,Cambridge,Waltham,Campbell,Palo Alto,Burlingame
funding_round_permalink,/funding-round/43d9cf8ed93378cc6aac829f091f150f,/funding-round/72a52518449d574376e2a5b893464874,/funding-round/908692accf565bd3192c763b6ae87b92,/funding-round/246696fae6c6412927b9aa5b3e6eee62,/funding-round/baa6d01ce2522730dfd892262a9b0a73
funding_round_type,venture,venture,undisclosed,venture,venture
funding_round_code,C,E,NaN,B,A


In [9]:
#getting all (unique) columns of the datasets
uniqueCols = set() #or: instead of "{}", "set()"
for ele in dfAll:
    cols = ele.columns.tolist() #".tolist()" to make it a list (side note: ".values" to make it a numpy.ndarray)
    
    uniqueCols.update(cols) #use ".add()" for elements, ".update()" for lists 

In [10]:
#sorting the columns alphabetically
tempList = list(uniqueCols)
tempList.sort()
uniqueCols = tempList
uniqueCols

['acquired_at',
 'acquired_month',
 'acquirer_category_list',
 'acquirer_city',
 'acquirer_country_code',
 'acquirer_name',
 'acquirer_permalink',
 'acquirer_region',
 'acquirer_state_code',
 'category_list',
 'city',
 'company_category_list',
 'company_city',
 'company_country_code',
 'company_name',
 'company_permalink',
 'company_region',
 'company_state_code',
 'content',
 'country_code',
 'first_funding_at',
 'founded_at',
 'funded_at',
 'funding_round_code',
 'funding_round_permalink',
 'funding_round_type',
 'funding_rounds',
 'funding_total_usd',
 'homepage_url',
 'investor_city',
 'investor_country_code',
 'investor_name',
 'investor_permalink',
 'investor_region',
 'investor_state_code',
 'last_funding_at',
 'month_str',
 'name',
 'permalink',
 'price_amount',
 'price_currency_code',
 'raised_amount_usd',
 'region',
 'state_code',
 'status',
 'value',
 'year_str']

# Seeing the columns in the datasets

In [11]:
def datasetsCols(dfList, datasetNames, pad=""):
    """
    Takes dataframes and returns a dataframe (df) with each column having a df's
    column names. The "pad" is to make sure all lists of columns have 
    the same length padded out with "pad"
    Example
    pad = "XX", datasetNames = ["cars", "planes"]
    dfList = [carsDF, planesDF]
    returned data frame:
            cars            planes
    0       numOfWheels     numOfWings
    1       manufacturer    manufacturer
    2       make            XX
    """
    cols = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        dictCsvs.update({datasetNames[i] : cols[i]})

    return pd.DataFrame(dictCsvs)

In [12]:
lstCsvNames = ["acquisitions", "additions", "companies", "investments", "rounds"]

dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,permalink,company_permalink,company_permalink
1,company_name,month_str,name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,country_code,company_city,company_city
7,acquirer_permalink,,state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,region,investor_name,funding_round_type
9,acquirer_category_list,,city,investor_country_code,funding_round_code


# Data Cleaning

First, lets change the columns in the `companies` dataset <br>
to be the similar to all other datasets:

In [13]:
companies.rename(columns = {'permalink' : 'company_permalink', 
                            'name' : 'company_name',
                            'category_list' : 'company_category_list',
                            'country_code' : 'company_country_code',
                            'state_code' : 'company_state_code',
                            'region' : 'company_region',
                            'city' : 'company_city'}, inplace = True)

In [14]:
dfAll[2] = companies
dfAllCols = datasetsCols(dfAll, lstCsvNames)
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


Let's start by analyzing `additions` dataset, as it has unusual format of columns

## Removing "additions" Dataset

In [15]:
additions.sample(5)

,content,month_str,year_str,value
136,Acquisitions,2002-07,2002,10
1493,Rounds: debt_financing,2014-04,2014,139
315,IPOs,1979-06,1979,2
252,Acquisitions,2012-03,2012,136
2166,Rounds: venture,2012-02,2012,235


In [16]:
additions["content"].unique()

array(['Acquisitions', 'IPOs', 'Profiles', 'Profiles: Organizations',
       'Profiles: People', 'Profiles: Products', 'Rounds',
       'Rounds: angel', 'Rounds: convertible_note',
       'Rounds: debt_financing', 'Rounds: equity_crowdfunding',
       'Rounds: grant', 'Rounds: non_equity_assistance',
       'Rounds: post_ipo_debt', 'Rounds: post_ipo_equity',
       'Rounds: private_equity', 'Rounds: product_crowdfunding',
       'Rounds: secondary_market', 'Rounds: seed', 'Rounds: undisclosed',
       'Rounds: venture'], dtype=object)

In [17]:
dfAllCols

,acquisitions,additions,companies,investments,rounds
0,company_permalink,content,company_permalink,company_permalink,company_permalink
1,company_name,month_str,company_name,company_name,company_name
2,company_category_list,year_str,homepage_url,company_category_list,company_category_list
3,company_country_code,value,company_category_list,company_country_code,company_country_code
4,company_state_code,,funding_total_usd,company_state_code,company_state_code
5,company_region,,status,company_region,company_region
6,company_city,,company_country_code,company_city,company_city
7,acquirer_permalink,,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,,company_region,investor_name,funding_round_type
9,acquirer_category_list,,company_city,investor_country_code,funding_round_code


In [18]:
additions.shape

(2213, 4)

In [19]:
companies.shape, rounds.shape

((66368, 14), (114949, 12))

So apparently there are no columns that could be used to join with the other datasets. <br>
Furthermore, the "value" column is too vague to be useful. <br>
Therefore, the "additions" dataset will be discarded.

In [20]:
dfAll.pop(1)
lstCsvNames.pop(1)
len(dfAll), lstCsvNames

(4, ['acquisitions', 'companies', 'investments', 'rounds'])

# Data Cleaning (Cont.)

## Checking All Missing Values

In [21]:
def datasetsNulls(dfList, datasetNames, nullCol="nulls_", pad=""):
    """
    Use this when you want to display a column of column names,
    then a column of the null values, and repeat that for each dataset.
    Returns a dataframe
    """
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.isnull().sum().tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[nullCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [22]:
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,1,company_name,1.0,company_name,1,company_name,1.0
2,company_category_list,4984,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4669,company_category_list,3148.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4949,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4911,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4911,company_country_code,6958.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,1,company_state_code,8547.0,investor_permalink,12,funding_round_permalink,0.0
8,acquirer_name,1,company_region,8030.0,investor_name,12,funding_round_type,0.0
9,acquirer_category_list,2389,company_city,8028.0,investor_country_code,30939,funding_round_code,83809.0


Alternatively, we can combine all columns to be the dataframe's index:

In [23]:
#Alternatively:
def datasetsNullsSameIndex(datasetsNames, *args):
    """
    Use this when multiple datasets have a lot of columns
    with the same names.
    Returns a dataframe with the index value being the 
    columns of all datasets
    """
    dic = {}
    for i, df in enumerate(args):
        dic[datasetsNames[i]] = df.isnull().sum()
    return pd.DataFrame(dic)

In [24]:
datasetsNullsSameIndex(lstCsvNames, *dfAll)

,acquisitions,companies,investments,rounds
acquired_at,13.0,NaN,NaN,NaN
acquired_month,15.0,NaN,NaN,NaN
acquirer_category_list,2389.0,NaN,NaN,NaN
acquirer_city,1360.0,NaN,NaN,NaN
acquirer_country_code,1171.0,NaN,NaN,NaN
acquirer_name,1.0,NaN,NaN,NaN
acquirer_permalink,1.0,NaN,NaN,NaN
acquirer_region,1360.0,NaN,NaN,NaN
acquirer_state_code,1436.0,NaN,NaN,NaN
company_category_list,4984.0,3148.0,3632.0,3410.0


### `dropna()` for columns with few missing values

Notice that there is one row that doesn't contain a company name, <br>
so let's validate that it is the same company across all datasets, <br>
so we can remove it:

In [25]:
datasetsCols(dfAll, lstCsvNames)

,acquisitions,companies,investments,rounds
0,company_permalink,company_permalink,company_permalink,company_permalink
1,company_name,company_name,company_name,company_name
2,company_category_list,homepage_url,company_category_list,company_category_list
3,company_country_code,company_category_list,company_country_code,company_country_code
4,company_state_code,funding_total_usd,company_state_code,company_state_code
5,company_region,status,company_region,company_region
6,company_city,company_country_code,company_city,company_city
7,acquirer_permalink,company_state_code,investor_permalink,funding_round_permalink
8,acquirer_name,company_region,investor_name,funding_round_type
9,acquirer_category_list,company_city,investor_country_code,funding_round_code


In [26]:
#company_permalink of each row that doesn't have a company name:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

['mendatio"']
['/organization/tell-it-in']
['/organization/tell-it-in']
['/organization/tell-it-in']


So apparently, the `acquisitions` dataset has a different permalink that has no company name, <br>
So we could remove these two companies, as the total number of companies are big enough to get the insight that we want: 

In [27]:
for df in dfAll:
    df.dropna(subset=['company_name'], inplace=True)
#Check:
for df in dfAll:
    print(df[df['company_name'].isnull()]['company_permalink'].values)

[]
[]
[]
[]


`dfAllNulls` dataframe also shows that there are missing values for `acquirer_name` and `investor_name` <br>
which can't be imputed, so we'll also remove them:

In [28]:
# Note: same logic applies for the permalinks
acquis.dropna(subset=['acquirer_permalink', 'acquirer_name'], inplace=True)
invests.dropna(subset=['investor_permalink', 'investor_name'], inplace=True)

Also, we'll remove the missing values for columns that have less than 100 `NaN`, <br>
as they're small in comparison to their respective datasets:

In [29]:
for df in dfAll:
    nulls = df.isnull().sum()
    cols = nulls[(nulls <= 100) & (nulls != 0)].index.tolist()
    df.dropna(subset=cols, inplace=True)
dfAllNulls = datasetsNulls(dfAll, lstCsvNames)
dfAllNulls

,acquisitions,nulls_1,companies,nulls_2,investments,nulls_3,rounds,nulls_4
0,company_permalink,0,company_permalink,0.0,company_permalink,0,company_permalink,0.0
1,company_name,0,company_name,0.0,company_name,0,company_name,0.0
2,company_category_list,4977,homepage_url,5058.0,company_category_list,3632,company_category_list,3410.0
3,company_country_code,4661,company_category_list,3147.0,company_country_code,10521,company_country_code,8678.0
4,company_state_code,4940,funding_total_usd,0.0,company_state_code,13431,company_state_code,10946.0
5,company_region,4902,status,0.0,company_region,12485,company_region,10167.0
6,company_city,4902,company_country_code,6956.0,company_city,12483,company_city,10164.0
7,acquirer_permalink,0,company_state_code,8545.0,investor_permalink,0,funding_round_permalink,0.0
8,acquirer_name,0,company_region,8028.0,investor_name,0,funding_round_type,0.0
9,acquirer_category_list,2387,company_city,8026.0,investor_country_code,30927,funding_round_code,83808.0


## Converting & Imputing Dates

We are currently interested in 'founded_at', 'first_funding_at' and 'last_funding_at'. Before doing operations on dates, they need to be converted first from 'Object' to 'DateTime'.

In [30]:
#Farah: plz impute (not drop) the columns' missing values related to dates in each dataset which are:
#founded_at.. and that's it :] 
# Suggestion: convert the dates of all the columns that have date values from "object" to datetime
#       Helpful link: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#converting-to-timestamps
# Then, make the founded_at a couple of days before first_funding_at

#Sincerely No. -Farah :)

In [31]:
companies['founded_at'] =pd.to_datetime(companies['founded_at'], errors = 'coerce')
companies['first_funding_at'] =pd.to_datetime(companies['first_funding_at'], errors = 'coerce')
companies['last_funding_at'] =pd.to_datetime(companies['last_funding_at'], errors = 'coerce')
companies.dtypes

company_permalink                object
company_name                     object
homepage_url                     object
company_category_list            object
funding_total_usd                object
status                           object
company_country_code             object
company_state_code               object
company_region                   object
company_city                     object
funding_rounds                    int64
founded_at               datetime64[ns]
first_funding_at         datetime64[ns]
last_funding_at          datetime64[ns]
dtype: object

After converting, we need to impute the missing values on the dates of when those companies where founded at. So, this will be done by first creating a new column that calculates how long it took to get the first funing. This will be done by subtracting the date of the founding from the date of the first funding.

In [32]:
companies['time_before_first_fund'] = companies['first_funding_at'] - companies['founded_at']
companies['time_before_first_fund']

0             NaT
1       -187 days
2             NaT
3        443 days
4       1665 days
           ...   
66363         NaT
66364   -194 days
66365   1308 days
66366    365 days
66367         NaT
Name: time_before_first_fund, Length: 66343, dtype: timedelta64[ns]

After converting, we need to impute the missing values on the dates of when those companies where founded at. So, this will be done by first creating a new column that calculates how long it took to get the first funing. This will be done by subtracting the date of the founding from the date of the first funding.

In [33]:
companies['time_before_first_fund'].fillna(companies['time_before_first_fund'].mean(), inplace = True)
companies['time_before_first_fund']

0       1370 days 03:49:18.015610624
1                -187 days +00:00:00
2       1370 days 03:49:18.015610624
3                  443 days 00:00:00
4                 1665 days 00:00:00
                    ...             
66363   1370 days 03:49:18.015610624
66364            -194 days +00:00:00
66365             1308 days 00:00:00
66366              365 days 00:00:00
66367   1370 days 03:49:18.015610624
Name: time_before_first_fund, Length: 66343, dtype: timedelta64[ns]

After calculating the mean and filling the nulls with it, now we can impute the missing dates in the 'founded_at' column by subtracting the time before first funding from the first funding date.

In [34]:
companies['founded_at'].fillna(companies['first_funding_at'] - companies['time_before_first_fund'], inplace = True)
companies['founded_at']

0       2011-04-05 20:10:41.984389376
1       2014-09-04 00:00:00.000000000
2       2010-04-30 20:10:41.984389376
3       2007-01-01 00:00:00.000000000
4       2010-01-01 00:00:00.000000000
                     ...             
66363   2008-06-30 20:10:41.984389376
66364   2012-05-13 00:00:00.000000000
66365   2011-01-01 00:00:00.000000000
66366   2014-01-01 00:00:00.000000000
66367   2009-12-30 20:10:41.984389376
Name: founded_at, Length: 66343, dtype: datetime64[ns]

In [35]:
#farah stop point here

## Checking All `dtypes`

In [36]:
def datasetsDtypes(dfList, datasetNames, dtypesCol="dtypes_", pad=""):
    cols = []
    nulls = []
    maxArrayLen = 0
    for i, df in enumerate(dfList):
        cols.append(df.columns.tolist())
        nulls.append(df.dtypes.tolist())
        maxArrayLen = max(maxArrayLen, len(cols[i]))
    
    dictCsvs = {}
    for i, df in enumerate(dfList):
        cols[i] += [pad] * (maxArrayLen - len(cols[i])) #padding the lists to make them have equal lengths 
        nulls[i] += [None] *  (maxArrayLen - len(nulls[i]))
        dictCsvs[datasetNames[i]] = cols[i]
        dictCsvs[dtypesCol + str(i+1)] = nulls[i]
    
    return pd.DataFrame(dictCsvs)

In [37]:
datasetsDtypes(dfAll, lstCsvNames)

,acquisitions,dtypes_1,companies,dtypes_2,investments,dtypes_3,rounds,dtypes_4
0,company_permalink,object,company_permalink,object,company_permalink,object,company_permalink,object
1,company_name,object,company_name,object,company_name,object,company_name,object
2,company_category_list,object,homepage_url,object,company_category_list,object,company_category_list,object
3,company_country_code,object,company_category_list,object,company_country_code,object,company_country_code,object
4,company_state_code,object,funding_total_usd,object,company_state_code,object,company_state_code,object
5,company_region,object,status,object,company_region,object,company_region,object
6,company_city,object,company_country_code,object,company_city,object,company_city,object
7,acquirer_permalink,object,company_state_code,object,investor_permalink,object,funding_round_permalink,object
8,acquirer_name,object,company_region,object,investor_name,object,funding_round_type,object
9,acquirer_category_list,object,company_city,object,investor_country_code,object,funding_round_code,object


In [38]:
#AISHA: see what you can do with the dtypes above (except dates, that's on Farah) ^^

In [39]:
#Aisha stop

## `rounds` Dataset

In [40]:
dfAllCols.iloc[:, -2:]

,investments,rounds
0,company_permalink,company_permalink
1,company_name,company_name
2,company_category_list,company_category_list
3,company_country_code,company_country_code
4,company_state_code,company_state_code
5,company_region,company_region
6,company_city,company_city
7,investor_permalink,funding_round_permalink
8,investor_name,funding_round_type
9,investor_country_code,funding_round_code


In [41]:
invests.shape, rounds.shape

((168634, 18), (114948, 12))

In [42]:
invests.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/organization/dt-capital-partners,DT Capital Partners,CHN,23,Shanghai,Shanghai,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0
1,/organization/004-technologies,004 Technologies,Software,USA,IL,"Springfield, Illinois",Champaign,/organization/venturecapital-de,VCDE Venture Partners,DEU,5,Frankfurt,Frankfurt,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,7/24/2014,NaN
2,/organization/01games-technology,01Games Technology,Games,HKG,NaN,Hong Kong,Hong Kong,/organization/cyberport-hong-kong,Cyberport Hong Kong,HKG,NaN,Hong Kong,Hong Kong,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,7/1/2014,41250.0
3,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/capital-one,Capital One,USA,VA,"Washington, D.C.",Mclean,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,B,11/9/2015,20000000.0
4,/organization/0xdata,H2O.ai,Analytics,USA,CA,SF Bay Area,Mountain View,/organization/nexus-venture-partners,Nexus Venture Partners,USA,CA,SF Bay Area,Menlo Park,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,5/22/2013,3000000.0


In [43]:
rounds.head()

,company_permalink,company_name,company_category_list,company_country_code,company_state_code,company_region,company_city,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,Media,IND,16,Mumbai,Mumbai,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,1/5/2015,10000000.0
1,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,10/14/2014,NaN
2,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,USA,DE,DE - Other,Delaware City,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,3/1/2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,1/30/2014,3406878.0
4,/organization/0-6-com,0-6.com,Curated Web,CHN,22,Beijing,Beijing,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,3/19/2008,2000000.0


In [44]:
len(invests["company_name"].unique()), len(rounds["company_name"].unique())

(44577, 66102)

Since there are companies in `rounds` dataset that are not in `investments` dataset, <br>
therefore we should keep `rounds` dataset for further analysis.

## `companies` Dataset

In [45]:
companies.sample(5).T

,33159,38926,24223,51590,30323
company_permalink,/organization/longfan-media,/organization/new-channel-online-school,/organization/group-ib,/organization/signalsense,/organization/keego-technologies
company_name,Longfan Media,New Channel Online School,Group-IB,SignalSense,MudWatt
homepage_url,http://www.longfanmedia.com,http://www.en51.com,http://www.group-ib.com/,http://www.signalsense.com,http://www.mudwatt.com/
company_category_list,News,Education,Cyber Security|Data Security|Fraud Detection,Network Security,Environmental Innovation
funding_total_usd,20000000,-,938000,4500000,60590
status,operating,operating,operating,operating,operating
company_country_code,CHN,CHN,RUS,USA,USA
company_state_code,30,22,48,WA,CA
company_region,Shenzhen,Beijing,Moscow,Seattle,SF Bay Area
company_city,Shenzhen,Beijing,Moscow,Seattle,Palo Alto


In [46]:
companies.isnull().sum()

company_permalink            0
company_name                 0
homepage_url              5058
company_category_list     3147
funding_total_usd            0
status                       0
company_country_code      6956
company_state_code        8545
company_region            8028
company_city              8026
funding_rounds               0
founded_at                   1
first_funding_at             3
last_funding_at              0
time_before_first_fund       0
dtype: int64

we will need to remove the none values of the following columns <br>
based on the question that we are trying to gain insight for: <br>
1.	Can the factors that affect a start-up’s growth be determined?
    -	For this: We are initially interested in <br> `funding_total_usd`, `status`, `funding_rounds`, `first_funding_at`, and `last_funding_at`
    -   So we could remove the 24 null values of `first_funding_at` since they are so small <br>
        compared to the rest of the dataset
2.  Do taxes play an important factor in deciding a start-up’s success?



test